In [1]:
import pandas as pd
import folium
import branca.colormap as cm

In [2]:
# Load datasets
championship_data = pd.read_csv('Championship_data.csv', encoding='ISO-8859-1')
geo_data = pd.read_csv('uscities.csv')

# Clean city names in both datasets
championship_data['City'] = championship_data['City'].str.strip()
geo_data['city'] = geo_data['city'].str.strip()

# Align the State Name to avoid mismatch
city_mappings = {
    'Golden State': 'San Francisco',
    'Minnesota': 'Minneapolis',
    'Utah': 'Salt Lake City',
    'Indiana': 'Indianapolis'
}
championship_data['City'] = championship_data['City'].replace(city_mappings)

In [3]:
# Merge the datasets
geo_col = ['city', 'lat', 'lng']
champ_col = ['City', 'NBA Finals wins by team (1946 - 2024)', 'Number of Championship']
merged_data = pd.merge(
    geo_data[geo_col],
    championship_data[champ_col],
    left_on='city',
    right_on='City',
    how='inner'
)

In [4]:
# Create a cool-warm sequential colormap
# Map color credit to http://vrl.cs.brown.edu/color
max_championships = merged_data['Number of Championship'].max()
colormap = cm.LinearColormap(
    colors = ['#4575b4', '#74add1', '#e0f3f8', '#fdae61', '#d73027'],
    vmin = 0,
    vmax = max_championships
)

In [5]:
# Create map filter the long and lat to US Territory
# Credit to https://www.geodatos.net/en/coordinates/united-states for the filter
interactive_map = folium.Map(location = [37.09, -95.71], zoom_start = 4)

In [6]:
# Add markers
for index, row in merged_data.iterrows():
    city = row['city']
    championships = int(row['Number of Championship'])
    team = row['NBA Finals wins by team (1946 - 2024)']
    latitude = row['lat']
    longitude = row['lng']
    
    color = colormap(championships)
    
    popup_text = f"""
    <div style='font-family: Arial; font-size: 12px;'>
        <b>City:</b> {city}<br>
        <b>Team:</b> {team}<br>
        <b>Championships:</b> {championships}
    </div>
    """
    
    folium.CircleMarker(
        location = [latitude, longitude],
        radius = 10 + (championships * 0.5),
        popup = folium.Popup(popup_text, max_width = 200),
        color = color,
        fill = True,
        fill_color = color,
        fill_opacity = 0.7,
        weight = 2
    ).add_to(interactive_map)

In [7]:
# Add the colormap to the map
colormap.add_to(interactive_map)
colormap.caption = 'NBA Championships (1946-2024)'

# Display the map
interactive_map